In [ ]:
import os
from typing import List

import equinox as eqx
import jax.numpy as jnp
from jaxtyping import Float, Array

from tunax import Obs, Database, FittableParameter, FittableParametersSet, Trajectory, Fitter

# create the database
observations = []
weights = []
variables = []
space_step = 4
for timeframe in timeframe_list:
    for forcing in forcing_list:
        filename = os.path.join(
            'k-epsilon_Wagner_LES', 'Wagner_LES',f'{timeframe}_hour_suite',
            f'{space_step}m', f'{forcing}_with_tracer_instantaneous_statistics.jld2')
        obs = Obs.from_jld2(filename, var_names, None, dims, eos_tracers='b', do_pt=True)

        # case corrections
        obs: Obs = eqx.tree_at(lambda t: t.case.ustr_sfc, obs, -obs.metadatas['u_str'])
        obs: Obs = eqx.tree_at(lambda t: t.case.b_forcing, obs, (0., -obs.metadatas['b_str']))
        obs: Obs = eqx.tree_at(lambda t: t.case.grav, obs, 9.80665)

        # passive tracer forcing
        omega_p = 1/obs.metadatas['pt_timescale']
        lambda_c = obs.metadatas['pt_width']
        zc = obs.metadatas['pt_depth']
        l_z = obs.trajectory.grid.hbot
        wrapped_forcing_pt = lambda z: forcing_passive_tracer(z, omega_p, lambda_c, -zc, l_z)
        obs: Obs = eqx.tree_at(lambda t: t.case.pt_forcing, obs, wrapped_forcing_pt)
        obs: Obs = eqx.tree_at(lambda t: t.case.pt_forcing_type, obs, 'constant', is_leaf=lambda x: x is None)

        # sunny forcing
        if forcing == 'strong_wind_and_sunny':
            jb = obs.metadatas['sunny_flux']
            eps1 = 0.6
            lambda1 = 1.
            lambda2 = 16.
            b_str = obs.case.b_forcing[1]
            wrapped_forcing_b_sunny = lambda z: forcing_buoyancy_sunny(z, jb, eps1, lambda1, lambda2)
            obs: Obs = eqx.tree_at(lambda t: t.case.b_forcing, obs, wrapped_forcing_b_sunny)
            obs: Obs = eqx.tree_at(lambda t: t.case.b_forcing_type, obs, 'constant', is_leaf=lambda x: x is None)
        
        # weight of the observation
        if forcing == 'free_convection':
            weights.append(1.)
            variables.append(['b', 'pt'])
        elif forcing in ['strong_wind_no_rotation', 'strong_wind_and_sunny']:
            weights.append(2/3)
            variables.append(['b', 'pt', 'u'])
        else:
            weights.append(1/2)
            variables.append(['b', 'pt', 'u', 'v'])
        observations.append(obs)
metadatas = {'weights': weights, 'variables': variables}
database = Database(observations, metadatas)

# fonction de normalisation d'un array
def norm_array(x: Float[Array, 'nz']):
    return (x-x.mean())/x.std()

# indices de la zone à sélectionner (pour le space step donné)
grid = database.observations[0].trajectory.grid
i_bot = jnp.searchsorted(grid.zr, grid.hbot/3, side='right')
i_top = jnp.searchsorted(grid.zr, -4, side='left')

# fonction coût
def loss(scm_set: List[Trajectory], database: Database):
    cost = 0.
    for i_obs in range(1):#len(scm_set)-34):
        traj_scm = scm_set[i_obs]
        traj_obs = database.observations[i_obs].trajectory
        for var in database.metadatas['variables'][i_obs]:
            var_scm = getattr(traj_scm, var)
            var_obs = getattr(traj_obs, var)
            end_scm = var_scm[-1, i_bot:i_top]
            end_obs = var_obs[-1, i_bot:i_top]
            norm_scm = norm_array(end_scm)
            norm_obs = norm_array(end_obs)
            var_cost = jnp.sum((norm_scm-norm_obs)**2)
            cost += database.metadatas['weights'][i_obs]*var_cost
    return cost

# parameters on the parameters to calibrate
c_eps1_par = FittableParameter(True, val=1.44)
c_eps2_par = FittableParameter(True, val=1.92)
c_eps3m_par = FittableParameter(True, val=.4)
c_eps3p_par = FittableParameter(True, val=1.)
sig_k_par = FittableParameter(True, val=1.)
sig_eps_par = FittableParameter(True, val=1.3)
coef_dico = {
    'c_eps1': c_eps1_par,
    'c_eps2': c_eps2_par,
    'c_eps3m': c_eps3m_par,
    'c_eps3p': c_eps3p_par,
    'sig_k': sig_k_par,
    'sig_eps': sig_eps_par
}
coef_fit_params = FittableParametersSet(coef_dico, 'k-epsilon')

plot_wag_nice

In [ ]:
titles = {
    'free_convection': 'free convection',
    'weak_wind_strong_cooling': 'weak wind\nstrong cooling',
    'med_wind_med_cooling': 'med wind\nmed cooling',
    'strong_wind_weak_cooling': 'strong wind\nweak cooling',
    'strong_wind': 'strong wind',
    'strong_wind_no_rotation': 'strong wind\nno rotation',
    'strong_wind_and_sunny': 'strong wind sunny\n no rotation'
}

def plot_wag(timeframe: int, space_step: int, tunax_version:str, experience_name: str):
    sp: subplot_2D_type = plt.subplots(3, 7, sharey=True, figsize=(6, 4))
    fig, [axs_b, axs_uv, axs_pt] = sp

    match tunax_version:
        case 'tunax':
            ObsClass = ObsOld
        case 'tunax_new':
            ObsClass = ObsNew
        case 'tunax_new_case_tracable':
            ObsClass = ObsNewTracable

    for i_forcing, forcing in enumerate(LES_FORCINGS):
        filename = os.path.join(LES_PATH, f'{timeframe}h', f'{space_step}m', f'{forcing}.jld2')
        wagner_obs = ObsClass.from_jld2(filename, LES_VAR_NAMES, None, LES_DIMS, eos_tracers='b', do_pt=True)
        wagner_traj = wagner_obs.trajectory
        output_path = os.path.join(OUTPUTS_PATH, experience_name)
        tunax_path = os.path.join(output_path, f'{timeframe}h', f'{space_step}m', f'{forcing}.nc')
        tunax_ds = xr.load_dataset(tunax_path)
        
        axs_b[i_forcing].plot(wagner_traj.b[0, :], wagner_traj.grid.zr, 'k', linewidth=1)
        axs_b[i_forcing].plot(wagner_traj.b[-1, :], wagner_traj.grid.zr, 'r')
        if experience_name == 'no_intermediate_outputs':
            axs_b[i_forcing].plot(tunax_ds.b, wagner_traj.grid.zr, 'g')
        else:
            axs_b[i_forcing].plot(tunax_ds.b[-1, :], wagner_traj.grid.zr, 'g')
        if i_forcing > 0:
            axs_uv[i_forcing].plot(wagner_traj.u[0, :], wagner_traj.grid.zr, 'k', linewidth=1)
            axs_uv[i_forcing].plot(wagner_traj.u[-1, :], wagner_traj.grid.zr, 'r')
            if experience_name == 'no_intermediate_outputs':
                axs_uv[i_forcing].plot(tunax_ds.u, wagner_traj.grid.zr, 'g')
            else:
                axs_uv[i_forcing].plot(tunax_ds.u[-1, :], wagner_traj.grid.zr, 'g')
            axs_uv[i_forcing].plot(wagner_traj.v[0, :], wagner_traj.grid.zr, 'k--', linewidth=1)
            axs_uv[i_forcing].plot(wagner_traj.v[-1, :], wagner_traj.grid.zr, 'r--')
            if experience_name == 'no_intermediate_outputs':
                axs_uv[i_forcing].plot(tunax_ds.v, wagner_traj.grid.zr, 'g--')
            else:
                axs_uv[i_forcing].plot(tunax_ds.v[-1, :], wagner_traj.grid.zr, 'g--')
        axs_pt[i_forcing].plot(wagner_traj.pt[0, :], wagner_traj.grid.zr, 'k', linewidth=1)
        axs_pt[i_forcing].plot(wagner_traj.pt[-1, :], wagner_traj.grid.zr, 'r')
        if experience_name == 'no_intermediate_outputs':
            axs_pt[i_forcing].plot(tunax_ds.pt, wagner_traj.grid.zr, 'g')
        else:
            axs_pt[i_forcing].plot(tunax_ds.pt[-1, :], wagner_traj.grid.zr, 'g')
        axs_b[i_forcing].set_title(titles[LES_FORCINGS[i_forcing]])
    axs_b[0].set_ylabel(r'$b []$')
    axs_uv[1].set_ylabel(r'$u, v \left[\mathrm m \cdot \mathrm s^{-1}\right]$')
    axs_pt[0].set_ylabel(r'passive tracer $[]$')
    axs_uv[0].plot([], [], 'g', label='tunax')
    axs_uv[0].plot([], [], 'r', label='LES')
    axs_uv[0].plot([], [], 'k', label='u')
    axs_uv[0].plot([], [], 'k--', label='v')
    axs_uv[0].text(-0.0, -40, f'time = {timeframe}h\nspace step = {space_step}m', fontsize=18, horizontalalignment='center')
    axs_uv[0].legend(loc='center')
    axs_uv[0].axis('off')
    axs_b[0].set_ylim(-200, 2)